In [16]:
from __future__ import print_function
from ortools.graph import pywrapgraph
import time
import os
import random as rng
import multiprocessing as mp
import numpy as np
import osmnx as ox
import matplotlib.pyplot as plt
ox.config(use_cache=True, log_console=False)
ox.__version__
import networkx as nx
from collections import defaultdict

start_time = time.process_time()

#initialise graph
G = ox.graph.graph_from_point([50.77543745,6.083803247551902],dist=3000,network_type='drive')
G = nx.convert_node_labels_to_integers(G)
node_info = G.nodes

# G_proj = ox.project_graph(G)
# G = ox.consolidate_intersections(G_proj, rebuild_graph=True, tolerance=1, dead_ends=False)
# ox.plot_graph(G2)


#number of entities
carrier_number = 10#rng.randint(100,200)
transportable_number = 10#rng.randint(100,200)


#functions to select random nodes
def create_random_node_points(number,Graph):
    node_list=[]
    for i in range(number):
        node_list.append(rng.randint(0,len(list(node_info))-1))
    return node_list

#create random carriers and transportables
carriers = create_random_node_points(carrier_number,G)
transportables = create_random_node_points(transportable_number,G)


dic = defaultdict(lambda: [])
    
dic['node_info'] = node_info

for i, carrier in enumerate(carriers):
    temp_dic = defaultdict(lambda: [])
    counter = 0
    for j, transportable in enumerate(transportables):

        if (ox.distance.euclidean_dist_vec(dic['node_info'][carrier]['y'],
                                           dic['node_info'][carrier]['x'],
                                           dic['node_info'][transportable]['y'],
                                           dic['node_info'][transportable]['x'])) <= 1:

            dic['connection_list'].append(j+1+len(carriers))
            temp_dic['end_numbers'].append(j)
            way = ox.shortest_path(G, carrier, transportable, weight='length')
            temp_dic['ways_to_transportable'].append(way)
            way_weight = 0

            for k in range(len(way)-1):
                way_weight += G[way[k]][way[k+1]][0]['length']

            dic['weight_list'].append(way_weight)
            counter+=1

    dic['end_list'].append(temp_dic['end_numbers'])
    dic['connection_number'].append(counter)
    dic['route_list'].append(temp_dic['ways_to_transportable'])
    
def plot_assigned_routes(Graph,car,trans,opt,route_l,end_l,car_num):
    nc=[]
    ns=[]
    for node in Graph.nodes():
        if node in car:
            nc.append('r')
            ns.append(80)
        elif node in trans:
            nc.append('g')
            ns.append(80)
        else:
            nc.append('w')
            ns.append(0)
        
    routes_to_plot = []
    for element in opt:
        routes_to_plot.append(route_l[element[0]-1][end_l[element[0]-1].index(element[1]-1)])

    ox.plot.plot_graph_routes(G2,routes_to_plot,route_colors='w', show=True, close=False, node_color=nc, node_size=ns)

    
    

In [17]:
ALI_G = nx.DiGraph()
counter = 0 
# weight_list = [int(x) for x in weight_list]

for j, trans in enumerate(transportables):
    ALI_G.add_node(trans, demand = 1)

for i, car in enumerate(carriers):
    ALI_G.add_node(car, demand = -1)



for i, car in enumerate(carriers):
#     G.add_node(car, demand = -1)   
    for j, trans in enumerate(transportables):
#         G.add_node(trans, demand = 1)
        ALI_G.add_edge(car, trans, weight=int(dic['weight_list'][counter]), capacity=1)
#         print(weight_list[counter])
        counter += 1

In [18]:
flowDict_alt = nx.min_cost_flow(ALI_G)
print(flowDict_alt)

{1268: {}, 578: {}, 653: {}, 1435: {}, 720: {}, 320: {}, 1266: {}, 646: {}, 104: {}, 649: {}, 268: {1268: 0, 578: 0, 653: 0, 1435: 0, 720: 0, 320: 0, 1266: 0, 646: 0, 104: 1, 649: 0}, 1092: {1268: 0, 578: 0, 653: 0, 1435: 0, 720: 0, 320: 0, 1266: 1, 646: 0, 104: 0, 649: 0}, 566: {1268: 0, 578: 0, 653: 1, 1435: 0, 720: 0, 320: 0, 1266: 0, 646: 0, 104: 0, 649: 0}, 1598: {1268: 0, 578: 0, 653: 0, 1435: 0, 720: 1, 320: 0, 1266: 0, 646: 0, 104: 0, 649: 0}, 233: {1268: 0, 578: 0, 653: 0, 1435: 0, 720: 0, 320: 1, 1266: 0, 646: 0, 104: 0, 649: 0}, 1001: {1268: 0, 578: 0, 653: 0, 1435: 0, 720: 0, 320: 0, 1266: 0, 646: 1, 104: 0, 649: 0}, 1574: {1268: 0, 578: 0, 653: 0, 1435: 0, 720: 0, 320: 0, 1266: 0, 646: 0, 104: 0, 649: 1}, 726: {1268: 0, 578: 1, 653: 0, 1435: 0, 720: 0, 320: 0, 1266: 0, 646: 0, 104: 0, 649: 0}, 1119: {1268: 0, 578: 0, 653: 0, 1435: 1, 720: 0, 320: 0, 1266: 0, 646: 0, 104: 0, 649: 0}, 1477: {1268: 1, 578: 0, 653: 0, 1435: 0, 720: 0, 320: 0, 1266: 0, 646: 0, 104: 0, 649: 0}}


In [5]:
flowDict = nx.min_cost_flow(G)
print(flowDict)

{614: {}, 1188: {}, 272: {}, 912: {}, 643: {}, 552: {}, 486: {}, 1463: {}, 134: {}, 728: {}, 24: {614: 1, 1188: 0, 272: 0, 912: 0, 643: 0, 552: 0, 486: 0, 1463: 0, 134: 0, 728: 0}, 87: {614: 0, 1188: 0, 272: 1, 912: 0, 643: 0, 552: 0, 486: 0, 1463: 0, 134: 0, 728: 0}, 1035: {614: 0, 1188: 0, 272: 0, 912: 0, 643: 0, 552: 0, 486: 1, 1463: 0, 134: 0, 728: 0}, 1071: {614: 0, 1188: 0, 272: 0, 912: 0, 643: 0, 552: 1, 486: 0, 1463: 0, 134: 0, 728: 0}, 138: {614: 0, 1188: 0, 272: 0, 912: 1, 643: 0, 552: 0, 486: 0, 1463: 0, 134: 0, 728: 0}, 347: {614: 0, 1188: 0, 272: 0, 912: 0, 643: 0, 552: 0, 486: 0, 1463: 0, 134: 1, 728: 0}, 1411: {614: 0, 1188: 0, 272: 0, 912: 0, 643: 0, 552: 0, 486: 0, 1463: 1, 134: 0, 728: 0}, 1272: {614: 0, 1188: 1, 272: 0, 912: 0, 643: 0, 552: 0, 486: 0, 1463: 0, 134: 0, 728: 0}, 149: {614: 0, 1188: 0, 272: 0, 912: 0, 643: 0, 552: 0, 486: 0, 1463: 0, 134: 0, 728: 1}, 1603: {614: 0, 1188: 0, 272: 0, 912: 0, 643: 1, 552: 0, 486: 0, 1463: 0, 134: 0, 728: 0}}


In [12]:
from collections import defaultdict
dic = defaultdict(lambda: [])
    
dic['node_info'] = node_info

for i, carrier in enumerate(carriers):
    temp_dic = defaultdict(lambda: [])
    counter = 0

    for j, transportable in enumerate(transportables):

        if (ox.distance.euclidean_dist_vec(dic['node_info'][carrier]['y'],
                                           dic['node_info'][carrier]['x'],
                                           dic['node_info'][transportable]['y'],
                                           dic['node_info'][transportable]['x'])) <= 1:

            dic['connection_list'].append(j+1+len(carriers))
            temp_dic['end_numbers'].append(j)
            if counter == 0:
                print(carriers[i], transportables[j])
#             way = ox.shortest_path(G, carrier, transportable, weight='length')
            way = ox.shortest_path(G, carriers[i], transportables[j], weight='length')
            print(way)
            temp_dic['ways_to_transportable'].append(way)
            way_weight = 0

            for k in range(len(way)-1):
                way_weight += G[way[k]][way[k+1]][0]['length']

            dic['weight_list'].append(way_weight)
            counter+=1

    dic['end_list'].append(temp_dic['end_numbers'])
    dic['connection_number'].append(counter)
    dic['route_list'].append(temp_dic['ways_to_transportable'])
    
def plot_assigned_routes(Graph,car,trans,opt,route_l,end_l,car_num):
    nc=[]
    ns=[]
    for node in Graph.nodes():
        if node in car:
            nc.append('r')
            ns.append(80)
        elif node in trans:
            nc.append('g')
            ns.append(80)
        else:
            nc.append('w')
            ns.append(0)
        
    routes_to_plot = []
    for element in opt:
        routes_to_plot.append(route_l[element[0]-1][end_l[element[0]-1].index(element[1]-1)])

    ox.plot.plot_graph_routes(G2,routes_to_plot,route_colors='w', show=True, close=False, node_color=nc, node_size=ns)

    
G = nx.DiGraph()
counter = 0 
# weight_list = [int(x) for x in weight_list]

for j, trans in enumerate(transportables):
    G.add_node(trans, demand = 1)

for i, car in enumerate(carriers):
    G.add_node(car, demand = -1)



for i, car in enumerate(carriers):
#     G.add_node(car, demand = -1)   
    for j, trans in enumerate(transportables):
#         G.add_node(trans, demand = 1)
        G.add_edge(car, trans, weight=int(dic['weight_list'][counter]), capacity=1)
#         print(weight_list[counter])
        counter += 1
    
flowDict_alt = nx.min_cost_flow(G)
print(flowDict_alt)

24 614
[24, 614]


KeyError: 0

In [8]:
transportables

[614, 1188, 272, 912, 643, 552, 486, 1463, 134, 728]

In [13]:
carriers

[24, 87, 1035, 1071, 138, 347, 1411, 1272, 149, 1603]

In [14]:
ox.shortest_path(G, 24, 614, weight='length')

[24, 614]